# Load an instance

In [ ]:
import lndb
from lndb import settings
from lndb._settings_store import instance_settings_file

## Load your own instance by name

If the user is the instance owner, just load the instance by name:

```
!lndb load mydata
```

In [ ]:
lndb.load("mydata")

In [ ]:
import os

assert settings.instance.storage.is_cloud == False
assert settings.instance.name == "mydata"
assert settings.instance.storage.root.as_posix() == f"{os.getcwd()}/mydata"
assert settings.instance.storage.cache_dir is None
assert settings.instance.db == f"sqlite:///{os.getcwd()}/mydata/mydata.lndb"

In [ ]:
# another test case, this time with a manually configured instance name

import os

lndb.load("mydata_instance_name")
assert settings.instance.storage.is_cloud == False
assert settings.instance.name == "mydata_instance_name"
assert settings.instance.storage.root.as_posix() == f"{os.getcwd()}/mydata_storage"
assert settings.instance.storage.cache_dir is None
assert (
    settings.instance.db
    == f"sqlite:///{os.getcwd()}/mydata_storage/mydata_instance_name.lndb"
)
lndb.delete("mydata_instance_name")

This also works for remote instances:

In [ ]:
# ensure that the locally cached env file is deleted
instance_settings_file("lndb-setup-ci", "testuser1").unlink()

In [ ]:
lndb.load("lndb-setup-ci")  # lndb load lndb-setup-ci

In [ ]:
assert settings.instance.storage.is_cloud == True
assert str(settings.instance.storage.root) == "s3://lndb-setup-ci"
assert str(settings.instance._sqlite_file) == "s3://lndb-setup-ci/lndb-setup-ci.lndb"

## Load instance from another owner

If you have the permission, you can load an instance from another owner. 

In [ ]:
lndb.load("testuser1/mydata")  # lndb load "testuser1/mydata"

Load an instance from wrong owner.

In [ ]:
assert lndb.load("testuser2/mydata") == "instance-not-exists"